In [30]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re
import sys
import time
from tqdm import tqdm

%matplotlib inline

In [31]:
documents = pd.read_json('documents.json').sort_index()

In [32]:
sys.getsizeof(documents)

119315201

In [33]:
try:
    full_documents = pd.read_json('full_documents.json').sort_index()
except:
    full_documents = documents.copy()
    full_documents['content'] = np.NaN

In [34]:
documents.head()

,country,lang,more,page,url,nav,title,date
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,习近平会见阿富汗总统加尼(2018-06-10),2018-06-10
2,联合国,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_联合国_相关新闻_,习近平就安南逝世向联合国秘书长致慰问电(2018-08-20),2018-08-20
3,墨西哥,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_北美洲_墨西哥_相关新闻_,王毅会见墨西哥第一副外长德伊卡萨(2017-05-03),2017-05-03
4,亚太经合组织（APEC）,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_亚太经合组织（APEC）_相关新闻_,习近平结束出席亚太经合组织第二十五次领导人非正式会议并对越南社会主义共和国、老挝人民民主共和...,2017-11-14


In [35]:
type(pd.to_datetime(documents.loc[2,'date']))

pandas._libs.tslibs.timestamps.Timestamp

In [36]:
full_documents.head()

,country,lang,more,page,url,nav,title,date,content
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06,None
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,习近平会见阿富汗总统加尼(2018-06-10),2018-06-10,None
2,联合国,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_联合国_相关新闻_,习近平就安南逝世向联合国秘书长致慰问电(2018-08-20),2018-08-20,None
3,墨西哥,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_北美洲_墨西哥_相关新闻_,王毅会见墨西哥第一副外长德伊卡萨(2017-05-03),2017-05-03,None
4,亚太经合组织（APEC）,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_亚太经合组织（APEC）_相关新闻_,习近平结束出席亚太经合组织第二十五次领导人非正式会议并对越南社会主义共和国、老挝人民民主共和...,2017-11-14,None


In [37]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options, executable_path="geckodriver.exe")
print("Firefox Headless Browser Invoked")

Firefox Headless Browser Invoked


In [38]:
times = []

In [39]:
for i in tqdm(full_documents.loc[(full_documents['content'].isnull()) & ((full_documents['country'] == '挪威') | 
                                                          (full_documents['country'] == '瑞典') | 
                                                          (full_documents['country'] == '丹麦') | 
                                                          (full_documents['country'] == '芬兰') | 
                                                          (full_documents['country'] == '冰岛')),].index):
    t = time.time()
    
#     print('INDEX: ',i)
    try:
        url = full_documents.loc[i,'url']
        driver.get(url)
#         print(type(driver.page_source))
#         print(sys.getsizeof(driver.page_source))
        full_documents.loc[i,'content'] = driver.page_source
        full_documents.to_json('full_documents.json')

    except:
        print('Error')
    nt = time.time()
    d = nt - t
    times.append(d)
#     print("{:.2f} seconds".format(d))
#     print('\n')
    if d >= 10:
        time.sleep(np.random.randint(5,10))
    else:
        continue

0it [00:00, ?it/s]


In [40]:
full_documents.loc[full_documents['content'].notnull(),].shape

(11852, 9)

In [41]:
full_documents.loc[full_documents['content'].isnull(),].shape

(113893, 9)

In [42]:
full_documents.loc[(((full_documents['nav'].str.contains('英国')) |
                                   (full_documents['nav'].str.contains('德国')) |
                                    (full_documents['nav'].str.contains('欧盟'))
                                  )
                                  & (full_documents['content'].isnull())),].shape

(0, 9)

In [43]:
for i in tqdm(full_documents.loc[(((full_documents['nav'].str.contains('英国')) |
                                   (full_documents['nav'].str.contains('德国')) |
                                    (full_documents['nav'].str.contains('欧盟'))
                                  )
                                  & (full_documents['content'].isnull())),].index):
    t = time.time()
    
#     print('INDEX: ',i)
    try:
        url = full_documents.loc[i,'url']
        driver.get(url)
#         print(type(driver.page_source))
#         print(sys.getsizeof(driver.page_source))
        full_documents.loc[i,'content'] = driver.page_source
        full_documents.to_json('full_documents.json')

    except:
        print('Error')
    nt = time.time()
    d = nt - t
    times.append(d)
#     print("{:.2f} seconds".format(d))
#     print('\n')
    if d >= 10:
        time.sleep(np.random.randint(5,10))
    else:
        continue

0it [00:00, ?it/s]


In [44]:
full_documents.to_json('full_documents.json')

In [45]:
full_documents.loc[(full_documents['nav'].str.contains('欧洲') & (full_documents['content'].isnull())),].shape

(18173, 9)

In [46]:
full_documents.loc[(full_documents['nav'].str.contains('欧洲') & (full_documents['content'].notnull())),].shape

(10722, 9)

In [47]:
# for i in tqdm(full_documents.loc[full_documents['content'].isnull(),].index):
#     t = time.time()
    
#     print('INDEX: ',i)
#     try:
#         url = full_documents.loc[i,'url']
#         driver.get(url)
#         print(type(driver.page_source))
#         print(sys.getsizeof(driver.page_source))
#         full_documents.loc[i,'content'] = driver.page_source
#         full_documents.to_json('full_documents.json')

#     except:
#         print('Error')
#     nt = time.time()
#     d = nt - t
#     times.append(d)
#     print("{:.2f} seconds".format(d))
#     print('\n')
#     if d >= 10:
#         time.sleep(np.random.randint(5,10))
#     else:
#         continue

In [48]:
full_documents.loc[full_documents['content'].notnull(),].shape

(11852, 9)

In [49]:
full_documents.head()

,country,lang,more,page,url,nav,title,date,content
0,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,外交部副部长孔铉佑会见阿富汗离任驻华大使莫萨扎伊(2018-09-06),2018-09-06,None
1,阿富汗,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_亚洲_阿富汗_相关新闻_,习近平会见阿富汗总统加尼(2018-06-10),2018-06-10,None
2,联合国,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_联合国_相关新闻_,习近平就安南逝世向联合国秘书长致慰问电(2018-08-20),2018-08-20,None
3,墨西哥,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,首页_国家和组织_国家（地区）_北美洲_墨西哥_相关新闻_,王毅会见墨西哥第一副外长德伊卡萨(2017-05-03),2017-05-03,None
4,亚太经合组织（APEC）,zh,NaN,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gjhd...,首页_国家和组织_国际和地区组织_亚太经合组织（APEC）_相关新闻_,习近平结束出席亚太经合组织第二十五次领导人非正式会议并对越南社会主义共和国、老挝人民民主共和...,2017-11-14,None


In [50]:
full_documents.loc[full_documents['content'].isnull(),].shape

(113893, 9)

In [51]:
full_documents.loc[full_documents['content'].notnull(),].shape

(11852, 9)

In [52]:
full_documents.loc[(full_documents['nav'].str.contains('欧洲') & (full_documents['content'].isnull())),].shape

(18173, 9)

In [53]:
full_documents.loc[(full_documents['content'].isnull()) & ((full_documents['country'] == '挪威') | 
                                                          (full_documents['country'] == '瑞典') | 
                                                          (full_documents['country'] == '丹麦') | 
                                                          (full_documents['country'] == '芬兰') | 
                                                          (full_documents['country'] == '冰岛')),].index

Int64Index([], dtype='int64')

In [54]:
nordic = [
    '挪威',
    '瑞典',
    '丹麦',
    '芬兰',
    '冰岛'
]

In [55]:
np.unique(full_documents.loc[full_documents['content'].notnull(),'country'])

array(['丹麦', '乌克兰', '俄罗斯', '保加利亚', '克罗地亚', '冰岛', '列支敦士登', '匈牙利', '卢森堡',
       '圣马力诺', '塞尔维亚', '塞浦路斯', '奥地利', '安道尔', '希腊', '德国', '意大利', '拉脱维亚',
       '挪威', '捷克', '摩尔多瓦', '摩纳哥', '斯洛伐克', '斯洛文尼亚', '欧盟', '比利时', '法国',
       '波兰', '波黑', '爱尔兰', '爱沙尼亚', '瑞典', '瑞士', '白俄罗斯', '立陶宛', '罗马尼亚', '芬兰',
       '英国', '荷兰', '葡萄牙', '西班牙', '阿尔巴尼亚', '马其顿', '马耳他', '黑山'],
      dtype=object)

In [56]:
nordic_df = full_documents.loc[(full_documents['country'] == '挪威') | 
                      (full_documents['country'] == '瑞典') | 
                      (full_documents['country'] == '丹麦') | 
                      (full_documents['country'] == '芬兰') | 
                      (full_documents['country'] == '冰岛'),].copy()

In [57]:
nordic_df.shape

(2556, 9)

In [58]:
nordic_df.to_json('nordic_df.json')